In [1]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
# Import necessary libraries

import json
import joblib
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import scipy.sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import requests
url = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=3e78d9923545475397681c5e8a47383d&e=gTeFhD'
r = requests.get(url, allow_redirects=True)
url1 = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=a6e25a75d96e42139fc5b23ba7a7bc1b&e=S9FI43'
r = requests.get(url1, allow_redirects=True)

## Step 1: Join business dataset with reviews dataset. Tokenize reviews and convert to lowercase

In [6]:
# Store reviews for each business
import random
business_information = defaultdict(lambda: {"reviews": []})

with open("/content/drive/MyDrive/yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:
    for line in file:
        # Keep only 10% of the lines
        if random.random() <= 0.10:
            curr = json.loads(line)
            business_information[curr["business_id"]]["reviews"].append(curr["text"])

In [7]:
# Extract business-specific data and store in dictionary
with open("/content/drive/MyDrive/yelp_academic_dataset_business.json", "r", encoding="utf-8") as file:

    for line in file:

        curr = json.loads(line)
        business_information[curr["business_id"]]["name"] = curr["name"]
        business_information[curr["business_id"]]["address"] = curr["address"]
        business_information[curr["business_id"]]["city"] = curr["city"]
        business_information[curr["business_id"]]["state"] = curr["state"]
        business_information[curr["business_id"]]["latitude"] = curr["latitude"]
        business_information[curr["business_id"]]["longitude"] = curr["longitude"]
        business_information[curr["business_id"]]["stars"] = curr["stars"]
        business_information[curr["business_id"]]["categories"] = curr["categories"]

In [8]:
# Tokenize reviews and convert all tokens to lowercase
tokenized_business_information = defaultdict(lambda: {"reviews": []})

for id_ in business_information.keys():

    for review in business_information[id_]["reviews"]:

      if random.random() <= 0.10:

        tokenized_business_information[id_]["reviews"].append([word.lower() for word in word_tokenize(review)])

In [9]:
# Write joined, tokenized business/review data to a file
with open('business_with_reviews.json', 'w') as f:
    for key, value in tokenized_business_information.items():
        json.dump({key: value}, f)
        f.write('\n')

## B+

2 Problems:
1. Can't extract user lat or lon
2. Have to hardcode every bordering state

In [10]:
user_state = "CA"
user_lat = 37.77   # San Francisco
user_lon = -122.42
# Hardcode every brodering state
bordering_states = {
    'AL': ['FL', 'GA', 'MS', 'TN'],
    'AK': [],
    'AZ': ['CA', 'CO', 'NM', 'NV', 'UT'],
    'AR': ['LA', 'MO', 'MS', 'OK', 'TN', 'TX'],
    'CA': ['AZ', 'NV', 'OR'],
    'CO': ['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY'],
    'CT': ['MA', 'NY', 'RI'],
    'DE': ['MD', 'NJ', 'PA'],
    'FL': ['AL', 'GA'],
    'GA': ['AL', 'FL', 'NC', 'SC', 'TN'],
    'HI': [],
    'ID': ['MT', 'NV', 'OR', 'UT', 'WA', 'WY'],
    'IL': ['IA', 'IN', 'KY', 'MO', 'WI'],
    'IN': ['IL', 'KY', 'MI', 'OH'],
    'IA': ['IL', 'MN', 'MO', 'NE', 'SD', 'WI'],
    'KS': ['CO', 'MO', 'NE', 'OK'],
    'KY': ['IL', 'IN', 'MO', 'OH', 'TN', 'VA', 'WV'],
    'LA': ['AR', 'MS', 'TX'],
    'ME': ['NH'],
    'MD': ['DE', 'PA', 'VA', 'WV'],
    'MA': ['CT', 'NH', 'NY', 'RI', 'VT'],
    'MI': ['IN', 'OH', 'WI'],
    'MN': ['IA', 'ND', 'SD', 'WI'],
    'MS': ['AL', 'AR', 'LA', 'TN'],
    'MO': ['AR', 'IA', 'IL', 'KS', 'KY', 'NE', 'OK', 'TN'],
    'MT': ['ID', 'ND', 'SD', 'WY'],
    'NE': ['CO', 'IA', 'KS', 'MO', 'SD', 'WY'],
    'NV': ['AZ', 'CA', 'ID', 'OR', 'UT'],
    'NH': ['MA', 'ME', 'VT'],
    'NJ': ['DE', 'NY', 'PA'],
    'NM': ['AZ', 'CO', 'OK', 'TX', 'UT'],
    'NY': ['CT', 'MA', 'NJ', 'PA', 'VT'],
    'NC': ['GA', 'SC', 'TN', 'VA'],
    'ND': ['MN', 'MT', 'SD'],
    'OH': ['IN', 'KY', 'MI', 'PA', 'WV'],
    'OK': ['AR', 'CO', 'KS', 'MO', 'NM', 'TX'],
    'OR': ['CA', 'ID', 'NV', 'WA'],
    'PA': ['DE', 'MD', 'NJ', 'NY', 'OH', 'WV'],
    'RI': ['CT', 'MA'],
    'SC': ['GA', 'NC'],
    'SD': ['IA', 'MN', 'MT', 'ND', 'NE', 'WY'],
    'TN': ['AL', 'AR', 'GA', 'KY', 'MO', 'MS', 'NC', 'VA'],
    'TX': ['AR', 'LA', 'NM', 'OK'],
    'UT': ['AZ', 'CO', 'ID', 'NV', 'NM', 'WY'],
    'VT': ['MA', 'NH', 'NY'],
    'VA': ['KY', 'MD', 'NC', 'TN', 'WV'],
    'WA': ['ID', 'OR'],
    'WV': ['KY', 'MD', 'OH', 'PA', 'VA'],
    'WI': ['IA', 'IL', 'MI', 'MN'],
    'WY': ['CO', 'ID', 'MT', 'NE', 'SD', 'UT']
}

In [11]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    angle = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    dist = 2 * asin(sqrt(angle))
    return dist * 6371

In [12]:
def extract_2nd(a):
    return a[1]

In [13]:
# def rank_by_location(query, state_u, lat_u, lon_u):
#     restaurant_values = []
#     allowed_states = [state_u] + bordering_states.get(state_u, [])

#     for bid, info in business_information.items():
#         # Check if any info is empty
#         lat, lon = info.get("latitude"), info.get("longitude")
#         if lon is None or lat is None:
#             continue
#         if "reviews" not in info or not info["reviews"]:
#             continue
#         # Check if the state is close, if not skip
#         if info.get("state") not in allowed_states:
#             continue

#         distance = haversine(lat_u, lon_u, lat, lon)
#         value = info["stars"]

#         # Find distance
#         if distance > 5:
#             # Only penalize the portion over 5km
#             value = value - 0.005 * (distance - 5)
#         restaurant_values.append((bid, value))

#     return sorted(restaurant_values, key=extract_2nd, reverse=True)[:10]

In [14]:
# results = rank_by_location("vegan", user_state, user_lat, user_lon)

# for n, value in results:
#     name = business_information[n]["name"]
#     print(f"{name} ({n}): score = {value:.2f}")

## Step 2: Output scores for TFIDF ranker

In [15]:
# Read in business reviews from JSON file
business_reviews = dict()

with open('business_with_reviews.json', 'r') as f:

    for line in f:

        record = json.loads(line)
        business_reviews.update(record)

In [16]:
# Test one query on the entire dataset, check results
business_ids = list(business_reviews.keys())
corpus = []
for id_ in business_ids:

    # Each business review is condensed down into a single, long sentence
    # so that TFIDF and other embedding techniques could be applied
    corpus.append(" ".join(" ".join(tokens) for tokens in business_reviews[id_]["reviews"]))

In [17]:
# Create TFIDF vector and train vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Persist vector and model
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)

In [18]:
# Reinitialize vectorizer and tfidf_matrix if session is lost
vectorizer = joblib.load('tfidf_vectorizer.joblib')
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")

In [19]:
# Transform query so that it fits into TFIDF vector space
query = "cafe with great lattes in san francisco"
query_vec = vectorizer.transform([query])

# Use cosine similarity to find most similar document for the query
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

In [20]:
# Match up business ids with similarity scores
scores = []

n = len(business_ids)
for i in range(n): scores.append((business_ids[i], similarities[i]))

# Sort based on score; sort in descending order
scores.sort(key = lambda x: x[1], reverse=True)

In [21]:
# Print out top 10 results
for id, _ in scores[:10]:

    print(business_information[id]["name"])

Backyard Beans Coffee Company
Tiparos Thai Restaurant
Morton's The Steakhouse
City of New Orleans
Urban China
The Outbound Station
Chappy's On Church
The Library Coffeehouse
San Francisco Style Sourdough Eatery - Boise
Quaker City Coffee


## Step 3: Create/Store Embeddings and Use FAISS to generate scores, Compare to token-matching ranker

In [22]:
!pip install torch

In [23]:
import torch

In [24]:
# Use transformer architecture to create dense vector embeddings
# from query and restaurant reviews
# all = general purpose transformer
# MiniLM = lightweight transformer similar to BERT
# L6 = six layers
# v2 = version two
model = SentenceTransformer("all-MiniLM-L6-v2")
subset_corpus = random.sample(corpus, int(0.01 * len(corpus)))
# Create embeddings from the reviews
embeddings = model.encode(
    subset_corpus,                      # List of documents
    batch_size=16,               # Tune this value (16–64 for Colab/CPU)
    convert_to_numpy=True,      # Needed for FAISS
    show_progress_bar=True,     # See how long it takes
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

dimension = embeddings.shape[1]

# Create FAISS index for similarity search
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [25]:
# Persist index to file
faiss.write_index(index, "business_index.faiss")

In [26]:
# Read index back from file if session is lost
index = faiss.read_index("business_index.faiss")

In [27]:
# Check results of test query
query = "cafe with great lattes in san francisco"
query_vec = model.encode([query], convert_to_numpy=True)

# Get top k results
k = 50
D, I = index.search(query_vec, k)
top_businesses = [business_ids[i] for i in I[0]]

for id in top_businesses:

    print(business_information[id]["name"])

Taqueria Cuernavaca
AJ's Fine Foods
AMC Plymouth Meeting Mall 12
Bacchanal Fine Wine & Spirits
The Drafting Room Tap Room & Grill
Pep Boys
Copabanana
Loews Philadelphia Hotel
Osaka Sushi
KRĒM
JW Marriott Indianapolis
Ice Cream 504
Al's Finger Licking Good Bar-B-Que
Johnny Rockets
Gaylord Opryland Resort & Convention Center
Yuengling Brewing Company of Tampa
Sushi Tyme
Bitar's
Sushi Rose
Chase Restaurant
Wei Asian Cafe
Firehouse Bar & Grill St. Louis
Bier Brewery and Tap Room
Casino Del Sol
Kien Giang
City Putt
Sierra Car Care
Plat 99
The Blue Duck
Dmitri's
The Wine Loft
Le Peep
The Pancake Pantry
Steamboat Natchez
Cafe Beignet on Bourbon Street
Helena Avenue Bakery
Ava
Gray Brothers Cafeteria
Makiman Sushi
Sbraga
Side Splitters Comedy Club
Sloppy Hoggs Roed Hus
Livery - Indianapolis
Jimboy's Tacos
Ruth's Chris Steak House
101 Taiwanese Cuisine
Love Nail & Spa
Hellas Restaurant
SharpShooter Indoor Shooting Range
Marathon


B+ with embeddings

In [33]:
def rank_by_location_with_embeddings(query, state_u, lat_u, lon_u, model, index, business_ids, business_info):
    allowed_states = [state_u] + bordering_states.get(state_u, [])

    # Encode query with SentenceTransformer
    query_vec = model.encode([query], convert_to_numpy=True)
    _, I = index.search(query_vec, 100)
    top_ids = [business_ids[i] for i in I[0]]

    restaurant_values_embed = []
    for bid in top_ids:
        info = business_info.get(bid, {})
        # Check if any info is empty
        lat, lon = info.get("latitude"), info.get("longitude")
        if not lat or not lon:
            continue
        if not info.get("reviews"):
            continue
        # Check if the state is close, if not skip
        if info.get("state") not in allowed_states:
            continue

        # Find distance
        distance = haversine(lat_u, lon_u, lat, lon)
        value = info["stars"]

        if distance > 5:
            # Only penalize the portion over 5km
            value = value - 0.005 * (distance - 5)

        restaurant_values_embed.append((bid, value))

    return sorted(restaurant_values_embed, key=extract_2nd, reverse=True)[:10]

In [34]:
# Example user location
user_state = "CA"
user_lat = 37.77
user_lon = -122.42

# Query and rank
results = rank_by_location_with_embeddings(
    "vegan burger", user_state, user_lat, user_lon,
    model, index, business_ids, business_information
)

# Print top results
for bid, score in results:
    print(f"{business_information[bid]['name']} ({bid}): score = {score:.2f}")

Coffeebar (CPFKi2lZJazP6IdtCdDDyg): score = 3.04
WinCo Foods (AbbIoHhBEMZndmIm07L4Og): score = 2.54
Chick-fil-A (ugNplY4SZqIfP417oUxahQ): score = 2.54
Kei Sushi (KBsLYznLfnJEY3RIBUZe0Q): score = 2.54
Johnny Rockets (ySSOZ_fz7wMhfjhWKzdR-w): score = 2.52
Roundin' Third (gEcrWktXXAR2YB2ruXg2Zw): score = 2.35
Sierra Car Care (Eqt-veWZaGVzInCr6zkNRQ): score = 2.04
Cal Taco (VeFfrEZ4iWaecrQg6Eq4cg): score = 1.85
Jimboy's Tacos (0GsYSmfpsUpnTnxYbH045A): score = 1.51
Las Trojes Mexican Restaurant (JYDd5sgn18DqDnTnq_tBRQ): score = 1.48
